# Homework 12 Solutions

## 2021

# WiFi

Take the CSV file wifi.csv and plot the location of the networks listed in the file, and plot a histogram of the number of networks for each town.  

In [ ]:
#### Plot Wifi Addresses
#
# Jeff Parker   April 2020

import csv
from collections import defaultdict
import matplotlib.pyplot as plt
import numpy as np

def import_file(fileName):
    "Takes a filename, returns a list of mile records"
    result = []

    try:                             # Try to open file: EFAP
        with open(fileName, 'rt') as f:
            reader = csv.reader(f)
            
            first = True
            # Get a new line from File
            for row in reader:
                if first:
                    first = False
                else:
                    result.append(row)
    except FileNotFoundError:
        print(f"No such file {fileName}")

    return result

In [ ]:
def plot_results(lst):
    """Plot the latitude and longitude"""
    # Now build lists
    x, y = [], []
    # for row in lst:
    for row in lst:
        x.append(float(row[0]))
        y.append(float(row[1]))
 
    plt.scatter(x, y, c='r', label='Wifi')
    
    plt.title('Wifi in Boston')
    plt.xlabel('Latitude')
    plt.ylabel('Longitude')
     
    # We could add a legend, but it isn't very useful
    # plt.legend()
    
    plt.show()

In [ ]:
lst = import_file('../../Data/wifi.csv')
plot_results(lst)

## Give a histogram of the towns and the number of WiFi networks

### While debugging, print out your list of towns: the address field needs grooming

In [ ]:
def count_towns(lst):
    """Take a list and count the towns"""
    towns = defaultdict(int)
    
    for row in lst:
        # Pull out the address
        address = row[2]
        parts = address.split(',')
        if (len(parts) < 2):
            raise ValueException(f"Unexpected address {address}")
            
        # The town is in the second segment
        town = parts[1].strip()
        if (len(parts) == 2):
            if ' MA' == town[-3:]:
                town = town[:-3]
        elif (len(parts) != 3):
            raise ValueException(f"Unexpected town in {address}")
            
        # Remove extra blanks
        town = town.replace('  ', ' ')
        
        # Count this town
        towns[town] += 1   
        
    # return dictionary counting Frequency
    return towns

In [ ]:
DEBUG = True

def plot_histogram(lst):
    towns = count_towns(lst)
    
    if DEBUG:
        print(f"{towns = }")

    # Now build lists
    x, y = [], []
    
    for town in towns:
        y.append(town)
        x.append(towns[town])
    
    # Horizontal Bar Chart
    plt.barh(y, x, align='center')

    plt.title('Wifi in Boston')
    plt.ylabel('Town')
    plt.xlabel('Number of WiFi networks')
    plt.show()

lst = import_file('../../Data/wifi.csv')
plot_histogram(lst)

# Employees

We wish to take an unordered list of Employees, and get a list sorted by Company and Id.

Everyone who works at 'Springfield Department of Motor Vehicles' should be in one group. 
Everyone who works at 'Springfield Nuclear Power' would be in another group, later in the list, 
and everyone who works from the Mafia would be in a group earlier in the list. 
Within each group, we want to see the low ID numbers before this high ones.

For this problem, we do not want you to write a sorting program. We are going to use Python's sort.   
To use Python's sort, you simply need to define the magic method __lt__() for the class Employee. 
Once that is done, calling Python's sorted() on a list of Employees will return a sorted list.

In [ ]:
class Person:

    def __init__(self, first, last):
        self.firstname = first.capitalize()
        self.lastname = last.capitalize()

        
    def __str__(self):
        return f"{self.firstname} {self.lastname}"


class Employee(Person):

    def __init__(self, first, last, company, id):
        # Call Superclass to set common information
        super().__init__(first, last)
        self.id = id
        self.company = company

        
    def __str__(self):
        # Call Superclass to dispaly common information
        return super().__str__() + f", {str(self.id)} at {self.company}"
    
  
    ### Add a definition of __lt__() - jdp
    def __lt__(self, other):
        # Use Tuple ordering
        return (self.company, self.id) < (other.company, other.id)

## Commentary

```python
    def __init__(self, first, last):
        self.firstname = first.capitalize()
        self.lastname = last.capitalize()
        
Employee('homer', 'simpson', 'Springfield Nuclear Power', 1005)    
```

## Unit Tests

In [ ]:
lst = [
    Employee('Homer', 'Simpson', 'Springfield Nuclear Power', 1005),
    Employee('Barney', 'Gumble', 'Plow King', 1),
    Employee('Clancy', 'Wiggum', 'Police Department', 1),
    Employee('Edna', 'Krabapple', 'Springfield Elementary School', 39),
    Employee('Seymour', 'Skinner', 'Springfield Elementary School', 1),
    Employee('Charles', 'Burns', 'Springfield Nuclear Power', 1),
    Employee('Waylon', 'Smithers', 'Springfield Nuclear Power', 2),
    Employee('Patty', 'Bouvier', 'Springfield Department of Motor Vehicles', 39),
    Employee('Selma', 'Bouvier', 'Springfield Department of Motor Vehicles', 38),
    Employee('Robert', 'Terwilliger', 'Channel 6', 31),
    Employee('Herschel', 'Krustofsky', 'Channel 6', 2),
    Employee('Lois', 'Pennycandy', 'Channel 6', 46),
    Employee('Johnny', 'Cevasco', 'Mafia', 2),
    Employee('Fat', 'Tony', 'Mafia', 1),
    Employee('Max', 'Legman', 'Mafia', 3 ),
    Employee('Louie', 'Walters', 'Mafia', 4)
    ]

for emp in lst:
    print(emp)
print('==========================')

# Sort the people
lst = sorted(lst)

# Check that the list is sorted
for first, second in zip(lst[:-1], lst[1:]):
    assert (first.company, first.id) < (second.company, second.id)

for emp in lst:
    print(emp)

# Phone Numbers

The "North American Numbering Plan" (NANP) is a telephone numbering system used by many countries in North America. All NANP-countries share the same international country code: `1`.

You are to create a class that takes a string and returns an object holding a valid NANP phone number.  

NANP numbers are ten-digit numbers consisting of a three-digit area code and a seven-digit local number. The first three digits of the local number are the "exchange code", and the four-digit number which follows is the "subscriber number".

The format is usually represented as (NXX)-NXX-XXXX where `N` is any digit from 2 through 9 and `X` is any digit from 0 through 9.

Your task is to clean up differently formatted telephone numbers by removing punctuation and the country code (1) if present.  If you are asked to create a phone number that does not meet the pattern above, you should throw a ValueError with a string explaining the problem: too many or too few digits, or the wrong digits.  

For example, the strings below 

```python
    +1 (617) 495-4024

    617-495-4024

    1 617 495 4024

    617.495.4024
```

should all produce an object that is printed as (617) 495-4024

## ValueErrors

Each of the following strings should produce a ValueError exception.  Each should have a different error message.

For example, for the third string, you could say:

```python
     raise ValueError('Invalid Country Code')
```

### Here are the 5 errors should be able to catch

```python
    +1 (617) 495-40247 has too many digits

    (617) 495-402 has too few digits

    +2 (617) 495-4024 has the wrong country code

    (017) 495-4024 has an illegal area code

    (617) 195-4024 has an illegal exchange code
```

## A Student's Solution

In [ ]:
class Phone(object):
    """A NANP phone number"""

    def __init__(self, phone_number: str):
        
        self.phone_number = phone_number
        
        pass
       
    def __str__(self) -> str:
        """Return a formated string with the phone number"""
        
        n1 = self.phone_number.replace("+", "")
        n2 = n1.replace("(", "")
        n3 = n2.replace(")", "")
        n4 = n3.replace("-", "")
        n5 = n4.replace(" ", "")
        n6 = n5.replace(".", "")

        if len(n6) > 11:
            raise ValueError ("Length is too long.")
            
        elif len(n6) < 10:
            raise ValueError ("Length is too short.")
            
        elif (n6[0] == "2") and (n6[1] != "2"):
            raise ValueError ("Wrong country code.")
            
        elif (n6[0] == "0"):
            raise ValueError ("Illegal area code.")
            
        elif (len(n6) == 11) and (n6[4:7] == "195"):
            raise ValueError ("Illegal exchange code.")
            
        elif (len(n6) == 10) and (n6[3:6] == "195"):
            raise ValueError ("Illegal exchange code.")
        
        elif len(n6) == 11:    
            return "(" + n6[1:4] + ") " + n6[4:7] + "-" + n6[7:]
        
        elif len(n6) == 10:    
            return "(" + n6[0:3] + ") " + n6[3:6] + "-" + n6[6:10]
        
        pass

## Commentary
```python
class Phone(object):
    """A NANP phone number"""

    def __init__(self, phone_number: str):      
        self.phone_number = phone_number      
        pass
```
First, we don't need the 'pass'.

Second, the student does not check the validity of the string here.  There is no way this can pass the Unit Tests for invalid strings.  

## Commentary
```python
     def __str__(self) -> str:
        """Return a formated string with the phone number"""
        
        n1 = self.phone_number.replace("+", "")
        n2 = n1.replace("(", "")
        n3 = n2.replace(")", "")
        n4 = n3.replace("-", "")
        n5 = n4.replace(" ", "")
        n6 = n5.replace(".", "")
```
The student validates the string in the dunder str() method.  This is too late to do much good.  

Further, this will perform the editing and checking each time we print the number.  It is better to do this once, when we create the object.  

The student has removed space, comma, '(', ',', ')', and '-'.  He did not remove '$', '#', or any other symbols.  He did not remove letters.  He removed space, but he did not remove tab, new line, or return.

It is hard to produce a full list of the many things we don't want.  It is easy to describe what we do want - the 10 digits.  It is simpler here, and safer, to filter for what we want than to filter out what we don't want.  

```python
    # Filter out non-digits
    lst = [ch for ch in phone_number if ch.isdigit()]
    
    # Turn the list into a string
    num = ''.join(lst)
```

## Commentary
```python
     elif (len(n6) == 10) and (n6[3:6] == "195"):
            raise ValueError ("Illegal exchange code.")     
```
Which exchange code is illegal?  The user may be a long way from the call: tell them what you saw.

It is true that 195 is an illegal exchange code.  So is 194, 193, and 197 other exchange codes.
It is just as easy to check if the first digit is 0 or 1. 

Catch all invalid codes, and explain the problem.

```python
     if (num[3] in '01'):
        raise ValueError(f'Exchange Code {num[3:6]} is invalid: should not start with {num[3]}')
```

## My Solution

In [ ]:
class Phone(object):

    def __init__(self, phone_number: str):

        # Prune out non-digits
        lst = [ch for ch in phone_number if ch.isdigit()]
    
        # Turn the list into a string
        num = ''.join(lst)

        # Check the length... 
        ln = len(num)
        if (ln != 10):
            # and trim if there it starts with 1
            if (ln == 11):
                if (num[0] == '1'):
                    num = num[1:]
                else:
                    raise ValueError('Invalid Country Code: ' + num[0])
            elif ln < 10:
                raise ValueError(f'String is to short: {str(ln)} digits')
            else:
                raise ValueError(f'String is to long: {str(ln)} digits')

        # Check special digits
        if (num[0] in '01'):
            raise ValueError(f'Area Code {num[:3]} is invalid: should not start with {num[0]}')
        if (num[3] in '01'):
            raise ValueError(f'Exchange Code {num[3:6]} is invalid: should not start with {num[3]}')

        self.number = num
        self.area_code = num[:3]

    # Return a formated string with the phone number
    def __str__(self) -> str:
        return f'({self.area_code}) {self.number[3:6]}-{self.number[6:]}'                         

## Unit Tests

In [ ]:
number = Phone("(223) 456-7890")
# print(number.__str__())
assert(number.__str__() == "(223) 456-7890")

# Shorthand for the three lines above
assert(Phone("(223) 456-7890").__str__() == "(223) 456-7890")


assert(Phone("+1 (617) 495-4024").__str__() == "(617) 495-4024")
assert(Phone("617-495-4024").__str__() == "(617) 495-4024")
assert(Phone("1 617 495 4024").__str__() == "(617) 495-4024")
assert(Phone("617.495.4024").__str__() == "(617) 495-4024")
assert(Phone("6174954024").__str__() == "(617) 495-4024")

## Illegal NANP numbers
try:
    number = Phone("+1 (617) 495-40247")
    assert(1 == 0)    # Should not get here
except ValueError:
    pass

try:
    number = Phone("(617) 495-402")
    assert(1 == 0)    # Should not get here
except ValueError:
    pass
 
try:
    number = Phone("+2 (617) 495-4024")
    assert(1 == 0)    # Should not get here
except ValueError:
    pass
    
try:
    number = Phone("(017) 495-4024")
    assert(1 == 0)    # Should not get here
except ValueError:
    pass
 
try:
    number = Phone("(617) 195-4024")
    assert(1 == 0)    # Should not get here
except ValueError:
    pass
   
print("Pass!")

In [ ]:
number = Phone("+1 (617) 495-40247")

In [ ]:
number = Phone("(617) 495-402")

In [ ]:
number = Phone("+2 (617) 495-4024")

In [ ]:
number = Phone("(017) 495-4024")

In [ ]:
number = Phone("(617) 195-4024")